# Discrete Region Representation

This notebook is a graphical illustration for *Discrete Region Representation*.

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.collections import LineCollection

# you may need to change the result directory depends on where you left the test results :)
# don't forget the trailing slash!
RESULT_DIR = "./result/"
# make sure the directory exists!
FIGURE_OUTPUT_DIR = "./figure/"
TEST_SETTING_FILENAME = "default-setting.txt"

# set to true to save all figures to a file
ENABLE_SAVE_FIGURE = False

def saveFigure(fig, filename):
	if ENABLE_SAVE_FIGURE:
		fig.savefig(FIGURE_OUTPUT_DIR + filename + ".pdf")

with open(RESULT_DIR + TEST_SETTING_FILENAME, "rt") as setting_file:
	print(setting_file.read())

## Visualisation

This section goes through the principle of discrete region representation, and how the formulae work to find area of each region, as discussed in the paper.

Start with defining some helper functions.

In [ ]:
# the number of region in the world
REGION_COUNT = 4

class SymbolTable:

	@staticmethod
	def position():
		return "$\\mathbf{p}$"

	@staticmethod
	def region(region):
		return "$g = " + str(region) + '$'

	@staticmethod
	def vectorRow(row):
		return "$i = " + str(row) + '$'

	@staticmethod
	def regionfield():
		return "$s(" + SymbolTable.position()[1:-1] + ")$"

	@staticmethod
	def vectorisedRegionfield():
		return "$\\overrightarrow{s}(" + SymbolTable.position()[1:-1] + ")$"

	@staticmethod
	def normalisedRegionDescriptorBlender():
		return "$\\hat{\\mathbf{b}}(" + SymbolTable.position()[1:-1] + ")$"

	@staticmethod
	def regionDescriptor(region):
		return "$d_{" + str(region) + "}(" + SymbolTable.position()[1:-1] + ")$"

	@staticmethod
	def blendedDescriptor():
		return "$d(" + SymbolTable.position()[1:-1] + ")$"

def createRng():
	RANDOM_SEED = 0x8B1F19C297C04D0F
	return np.random.Generator(np.random.Philox(RANDOM_SEED))

def generateRandomColour(rng, count):
	# we don't use `.random` directly because the upper range is open
	return rng.integers(0, 255, (count, 3), dtype = np.uint8, endpoint = True).astype(np.float32) / 0xFF

def makeSquareSubplot(total):
	# organise subplots given the total number of them so they approximate to a square
	axe_extent = int(np.sqrt(total))
	axe_extent_x = axe_extent
	axe_extent_y = int(total / axe_extent_x)
	return axe_extent_x, axe_extent_y

### Region Descriptor

We define a descriptor for each region to help us to understand how the convolution kernels compute region blending weights, and how the blended descriptor looks like using different approaches.

For simplicity, we use simple periodic functions to construct our region descriptor set. In reality, region descriptor can be anything, such as heightfield for different biomes.

In [ ]:
class RegionDescriptorSet:
	# control the amplitude and frequency of the periodic functions
	AMPLITUDE = 0.8
	FREQUENCY = np.pi * 1.195

	OUTPUT_DOMAIN = (-30.5, 30.5)
	OUTPUT_RANGE = (-5.5, 5.5)
	OUTPUT_PLOT_DOMAIN_MARGIN = 1.5
	OUTPUT_PLOT_RANGE_MARGIN = 0.65
	# the number of sample per region descriptor function
	OUTPUT_RESOLUTION = 175

	def __init__(this):
		rng = createRng()
		this.DescriptorColour = generateRandomColour(rng, REGION_COUNT)
		
		# we notice having functions with changing vertical offsets seem to be more noticeable
		# than changing amplitude or frequency when functions are later blended together
		this.Sample = np.linspace(*RegionDescriptorSet.OUTPUT_DOMAIN, RegionDescriptorSet.OUTPUT_RESOLUTION,
			dtype = np.float32)
		this.BaseDescriptor = RegionDescriptorSet.AMPLITUDE * \
			np.cos(this.Sample * RegionDescriptorSet.FREQUENCY, dtype = np.float32)

		this.AltitudeStep = np.linspace(*RegionDescriptorSet.OUTPUT_RANGE, REGION_COUNT, dtype = np.float32)

	def __str__(this):
		ret = str()
		ret += "Descriptor Colour: " + str(this.DescriptorColour) + '\n'
		ret += "Altitude Step: " + str(this.AltitudeStep)
		return ret

	def __getitem__(this, region):
		# applying region-specific vertical offset
		return this.BaseDescriptor + this.AltitudeStep[region]

	def composeRegionDescriptor(this):
		# a 2D matrix, where row is the region descriptor for each region,
		# and column is the position
		return this.BaseDescriptor[None, :] + this.AltitudeStep[:, None]

	@staticmethod
	def getLimit():
		y_padding = RegionDescriptorSet.AMPLITUDE + RegionDescriptorSet.OUTPUT_PLOT_RANGE_MARGIN
		xlim = (RegionDescriptorSet.OUTPUT_DOMAIN[0] - RegionDescriptorSet.OUTPUT_PLOT_DOMAIN_MARGIN,
			RegionDescriptorSet.OUTPUT_DOMAIN[1] + RegionDescriptorSet.OUTPUT_PLOT_DOMAIN_MARGIN)
		ylim = (RegionDescriptorSet.OUTPUT_RANGE[0] - y_padding,
			RegionDescriptorSet.OUTPUT_RANGE[1] + y_padding)
		return xlim, ylim

	def plotDescriptorSet(this, size):
		fig, axe = plt.subplots(*makeSquareSubplot(REGION_COUNT),
			figsize = size, constrained_layout = True)
		fig.suptitle("Region Descriptor Set")

		xlim, ylim = RegionDescriptorSet.getLimit()
		for region, a in enumerate(axe.flat):
			a.set_title(SymbolTable.region(region))
			a.set_xlabel(SymbolTable.position())
			a.set_ylabel(SymbolTable.regionDescriptor(region))
			a.set_xlim(*xlim)
			a.set_ylim(*ylim)
			
			a.plot(this.Sample, this[region], color = this.DescriptorColour[region])

		saveFigure(fig, "region-descriptor-set")

	def makeBlendedDescriptor(this, norm_rd_blender):
		# the normalised region descriptor blender shall be an array of column vectors of weights
		region_descriptor = this.composeRegionDescriptor()
		# a simple column-wise dot product
		return (norm_rd_blender.T * region_descriptor).sum(axis = 0)

	def plotBlendedDescriptor(this, norm_rd_blender, subtitle, size, colour):
		fig, axe = plt.subplots(1, 1, figsize = size, constrained_layout = True)
		axe.set_title("Blended Descriptor({})".format(subtitle))
		axe.set_xlabel(SymbolTable.position())
		axe.set_ylabel(SymbolTable.blendedDescriptor())
		
		blended_descriptor = this.makeBlendedDescriptor(norm_rd_blender)
		# we want to make each segment to have different colours based ono the blend factor
		# need to use line collection
		# each line only has one segment so we can create a gradient-like colour
		vertex = np.vstack((this.Sample, blended_descriptor), dtype = np.float32).T[:, None, :]
		segment = np.concatenate((vertex[:-1], vertex[1:]), axis = 1)

		weight = norm_rd_blender.T[:, :, None]
		colour = this.DescriptorColour[:, None, :]
		lc = LineCollection(segment, colors = (weight * colour).sum(axis = 0))
		
		xlim, ylim = RegionDescriptorSet.getLimit()
		axe.set_xlim(*xlim)
		axe.set_ylim(*ylim)
		axe.add_collection(lc)

		saveFigure(fig, "blended-descriptor-{}".format(subtitle))

region_descriptor_set = RegionDescriptorSet()
print(region_descriptor_set)

Take a look at the descriptor for each region. They will be used to compose the blended descriptor later.

In [ ]:
region_descriptor_set.plotDescriptorSet(size = (8, 4))

### Regionfield

To make things simpler for visualisation, we use 1D regionfield as an example. First, we shall generate a random regionfield function.

In [ ]:
class Regionfield:
	REGION_DOMAIN = (-25.5, 25.5)
	# control the number of variation in regionfield
	REGION_SEGMENT = 8
	# control the ratio of minimum width of each segment against their original length
	MIN_SEGMENT_WIDTH_FACTOR = 2.0
	
	def __init__(this):
		rng = createRng()
		# generate regionfield function
		this.RegionColour = generateRandomColour(rng, REGION_COUNT)

		dom_min, dom_max = Regionfield.REGION_DOMAIN
		# add one additional size because we need to determine the begin and end of each region later
		num_segment = Regionfield.REGION_SEGMENT + 1
		segment_width = (dom_max - dom_min) / num_segment
		min_segment_width = segment_width / Regionfield.MIN_SEGMENT_WIDTH_FACTOR
		
		regionfield_domain_segment = np.arange(dom_min, dom_max, segment_width, dtype = np.float32)
		# random jitter domain segment
		max_jitter = segment_width - min_segment_width
		regionfield_domain_segment += rng.uniform(-max_jitter, max_jitter, num_segment).astype(np.float32)
		regionfield_domain_segment = np.sort(regionfield_domain_segment)

		regionfield_range = rng.integers(0, REGION_COUNT,
			Regionfield.REGION_SEGMENT, dtype = np.uint8)
		# there is a chance adjacent region may be the same, it does not matter in practice at all
		# but for visualisation purposes we beautify by removing duplicate adjacent region
		# you can try to remove this `for loop` to see what will happen :P
		# you may need to change the seed a few times to see the ugliness
		for i in range(1, Regionfield.REGION_SEGMENT - 1):
			prev, curr, next = [regionfield_range[j] for j in [i - 1, i, i + 1]]
			while curr == prev or curr == next:
				curr = rng.integers(0, REGION_COUNT, dtype = np.uint8)
			
			regionfield_range[i] = curr

		this.RegionfieldRange = regionfield_range
		this.RegionfieldDomainBegin = regionfield_domain_segment[:-1]
		this.RegionfieldDomainEnd = regionfield_domain_segment[1:]

	def __str__(this):
		ret = str()
		ret += "Region Colour: \n" + str(this.RegionColour) + '\n'
		ret += "Regionfield Range: \n" + str(this.RegionfieldRange) + '\n'
		ret += "Regoinfield Domain Begin: \n" + str(this.RegionfieldDomainBegin) + '\n'
		ret += "Regoinfield Domain End: \n" + str(this.RegionfieldDomainEnd)
		return ret

	def __getitem__(this, p):
		# get region assigned to point; the input can be an array of 1D points
		return this.RegionfieldRange[this.getRegionIndex(p)]

	def getRegionIndex(this, p):
		# get region index of a point; the input can be an array of 1D points
		region_idx = np.searchsorted(this.RegionfieldDomainBegin, p)
		# clamp to edge addressing mode
		# numpy binary search returns the next index, so minus one
		return np.maximum(region_idx, 1) - 1

	def getRegionfieldDomainColour(this):
		return this.RegionColour[this.RegionfieldRange]

	def definiteIntegration(this, bound):
		# find the area under each vectorised region function, given a tuple of domain bound
		p_begin, p_end = bound
		curr_p = p_begin
		# the region index will be clamped to zero if begin bound is beyond the range
		curr_region_idx = this.getRegionIndex(p_begin)

		area = np.zeros(REGION_COUNT, dtype = np.float32)
		max_region_idx = this.RegionfieldRange.shape[0] - 1
		while curr_p < p_end:
			# if end bound is beyond the range, do a clamp to edge addressing
			bound_begin = curr_p
			if curr_region_idx > max_region_idx:
				curr_region = this.RegionfieldRange[-1]
				bound_end = p_end
			else:
				curr_region = this.RegionfieldRange[curr_region_idx]
				bound_end = np.minimum(this.RegionfieldDomainEnd[curr_region_idx], p_end)
			# height of vectorised function is 1
			area[curr_region] += bound_end - bound_begin

			# advance to the next iteration
			curr_p = bound_end
			curr_region_idx += 1
		return area

	def plotRegionfield(this, size):
		domain_colour = this.getRegionfieldDomainColour()
		# there are 2 y axes, one for the original regionfield, the other for vectorised regionfield
		# they both use the same plot, just with different tick labels on y-axis
		# first draw regionfield axis as usual
		fig, axe_rf = plt.subplots(1, 1, figsize = size, constrained_layout = True)
		
		axe_rf.set_title("Regionfield")
		axe_rf.set_xlabel(SymbolTable.position())
		axe_rf.set_ylabel(SymbolTable.regionfield())
		axe_rf.grid(True, which = "both", axis = "y")
		
		axe_rf.hlines(this.RegionfieldRange, this.RegionfieldDomainBegin, this.RegionfieldDomainEnd,
			color = domain_colour)
		axe_rf.set_yticks(np.arange(0, REGION_COUNT, dtype = np.uint8))

		for x in [this.RegionfieldDomainBegin, this.RegionfieldDomainEnd]:
			axe_rf.vlines(x, np.repeat(0, x.shape[0]), this.RegionfieldRange,
				color = domain_colour, linestyle = ':')

		# create a vectorised regionfield axis, make sure they have the same scale
		axe_rp = axe_rf.twinx()
		axe_rp.set_ylabel(SymbolTable.vectorisedRegionfield())
		axe_rp.set_yticks(axe_rf.get_yticks())
		axe_rp.set_ylim(axe_rf.get_ylim())

		one_hot = np.eye(REGION_COUNT, dtype = np.uint8)
		one_hot_label = ["$(" + ','.join(map(str, region)) + ")^{T}$" for region in one_hot]
		axe_rp.set_yticklabels(one_hot_label)

		saveFigure(fig, "regionfield")

	@staticmethod
	def setCommonBlenderAxisLabel(axe, region):
		axe.set_title(SymbolTable.vectorRow(region))
		axe.set_xlabel(SymbolTable.position())
		axe.set_ylabel(SymbolTable.normalisedRegionDescriptorBlender())

	@staticmethod
	def plotBoundaryAxis(axe, xlim):
		for y in [0.0, 1.0]:
			axe.hlines(y, *xlim, linewidth = 1.5, color = "dimgray")

	def generateRegionBlenderUnblended(this, p):
		# basically an array of column vectors of region weights
		p_region = this[p]
		diagonal = np.eye(REGION_COUNT, dtype = np.float32)
		return diagonal[p_region]

	def plotRegionBlenderUnblended(this, size):
		fig, axe = plt.subplots(*makeSquareSubplot(REGION_COUNT),
			figsize = size, constrained_layout = True)
		fig.suptitle("Normalised Region Descriptor Blender(Unblended)")

		# draw rectangular patches for each region
		patch = dict()
		for region, begin, end, colour in zip(
			this.RegionfieldRange,
			this.RegionfieldDomainBegin,
			this.RegionfieldDomainEnd,
			this.getRegionfieldDomainColour()
		):
			patch.setdefault(region, list()).append(
				patches.Rectangle((begin, 0), end - begin, 1,
					color = colour, fill = None, hatch = "\\\\", linewidth = 1.0))

		axe_boundary = (this.RegionfieldDomainBegin[0], this.RegionfieldDomainEnd[-1])
		for region, a in enumerate(axe.flat):
			Regionfield.setCommonBlenderAxisLabel(a, region)
			a.set_yticks((0.0, 1.0))
			a.set_ylim((-0.05, 1.05))

			# seems like matplot won't scale the axis automatically when we are drawing patches
			# so draw an axis line ourselves
			Regionfield.plotBoundaryAxis(a, axe_boundary)
			
			region_patch = patch.get(region)
			if region_patch:
				for p in region_patch:
					a.add_patch(p)

		saveFigure(fig, "region-descriptor-blender-unblended")

	def generateRegionBlenderImRA(this, p, kernel_radius):
		blender = np.zeros((p.shape[0], REGION_COUNT), dtype = np.float32)
		blender[0] = this.definiteIntegration((p[0] - kernel_radius, p[0] + kernel_radius))
		# we compute the blending weights using our optimising trick from the paper
		for i, (prev, next) in enumerate(zip(p, p[1:])):
			i += 1 # because we have already computed the first one

			blender[i] = blender[i - 1]
			blender[i] += this.definiteIntegration((prev + kernel_radius, next + kernel_radius))
			blender[i] -= this.definiteIntegration((prev - kernel_radius, next - kernel_radius))
		# use L1 norm because we want the sum of weight from all regions of the same *p* equals 1.0
		return blender / np.linalg.norm(blender, ord = 1, axis = 1, keepdims = True)

	def plotRegionBlenderImRA(this, p, kernel_radius, size):
		# Plotting the blender computed by ImRA is not continuous, unlike the Unblended kernel,
		# because computer cannot take an infinite number of samples.
		# Instead we plot by a small increment given by variable *p*.
		fig, axe = plt.subplots(*makeSquareSubplot(REGION_COUNT),
			figsize = size, constrained_layout = True)
		fig.suptitle("Normalised Region Descriptor Blender(ImRA)")
		
		blender = this.generateRegionBlenderImRA(p, kernel_radius)
		for region, a in enumerate(axe.flat):
			Regionfield.setCommonBlenderAxisLabel(a, region)
			
			region_weight = blender[:, region]
			a.fill_between(p, region_weight, facecolor = "none",
				edgecolor = this.RegionColour[region], hatch = "\\\\", linewidth = 1.0)
			# draw axis line to hide the bottom edge color
			Regionfield.plotBoundaryAxis(a, a.get_xlim())

		saveFigure(fig, "region-descriptor-blender-imra")

regionfield = Regionfield()
print(regionfield)

Let's take a look at what the regionfield looks like. It looks like a multi-step function that returns a positive integer given position. The integer value is the region assigend to that point.

In [ ]:
regionfield.plotRegionfield(size = (9, 3))

#### Unblended Kernel

We can use a filter kernel to compute region descriptor blender, which gives a weight for each region for every point in the domain that can be used to select region descriptor for the output.

The unblended implementation simply reuses the definition of vectorised regionfield (which is just a one-hot encoded column vector), such that the output uses the exact region descriptor for the region assigned to point $p$.

In [ ]:
regionfield.plotRegionBlenderUnblended(size = (8, 4))

The limitation of the unblended kernel becomes clear once we plot the blended descriptor with this region descriptor blender; recall that a blended descriptor is computed by weighing each region discriptor from the descriptor set with a normalised region descriptor blender. It is noticeable that the function given by this blended descriptor is not continuous.

The main reason for the abrupt transition from one region to the other is due to the binary weight given by the region descriptor blender.

In [ ]:
def drawBlendedDescriptor(blender, subtitle):
	region_descriptor_set.plotBlendedDescriptor(blender,
		subtitle = subtitle, size = (6, 2), colour = "crimson")

drawBlendedDescriptor(regionfield.generateRegionBlenderUnblended(region_descriptor_set.Sample), "Unblended")

#### Implicit Region Area Kernel (ImRA)

The vectorised regionfield is useful when we need to find area each region occupies. As noticed, the vectorised regionfield does two things:

- Regions are separated into their own functions, so areas from different regions are added up independently.
- As regionfield merely represents an integer identifier that has no mathematical meaning, each region gets a binary weight, and region is assigned with weight of one if the current position belongs to this region, or zero otherwise.

The area of each region is the area under the function bounded by vectorised regionfield of each region. Doing so allows us to generate a linear interpolation of region descriptors of surrounded regions to the current point $p$, thus giving a continous transition.

In [ ]:
RDK_KERNEL_RADIUS = 3.5

regionfield.plotRegionBlenderImRA(region_descriptor_set.Sample, RDK_KERNEL_RADIUS, size = (8, 4))

Similarly, we can give the blended descriptor from region descriptor blender computed by our ImRA. It is observable that this function is now continuous.

In [ ]:
drawBlendedDescriptor(regionfield.generateRegionBlenderImRA(region_descriptor_set.Sample, RDK_KERNEL_RADIUS), "ImRA")

### Regoinmap

Second, we will be showing different regoinmaps with different region densities, which will be used as our test setup.

In [ ]:
class Regionmap:
	MAP_DIMENSION = (64, 64)
	# the number of centroid on the Voronoi diagram
	CENTRIOD_COUNT = 9

	def __init__(this):
		rng = createRng()
		this.RegionColour = generateRandomColour(rng, REGION_COUNT)

		this.Random = rng.integers(0, REGION_COUNT, Regionmap.MAP_DIMENSION, dtype = np.uint8)

		# generate a voronoi diagram
		# this can be done by finding the nearest centroid to each pixel
		cell_index = np.indices(Regionmap.MAP_DIMENSION, dtype = np.float32).T
		# flip the image vertically to match the convention
		cell_index = np.flip(cell_index, axis = 0)[:, :, None, :]
		
		centroid = rng.uniform((0.0, 0.0), Regionmap.MAP_DIMENSION,
			(Regionmap.CENTRIOD_COUNT, 2)).astype(np.float32)
		# randomly assign each Voronoi cell with a region
		voronoi_region = rng.integers(0, REGION_COUNT, Regionmap.CENTRIOD_COUNT, dtype = np.uint8)

		cell_distance = np.linalg.norm(cell_index - centroid, ord = 2, axis = 3)
		closest_centroid_index = np.argmin(cell_distance, axis = 2)
		this.Voronoi = voronoi_region[closest_centroid_index]

	def __str__(this):
		ret = str()
		ret += "Region Colour: " + str(this.RegionColour)
		return ret
	
	def plotMap(this, axe, which_map):
		match(which_map):
			case "Random":
				map = this.Random
			case "Voronoi":
				map = this.Voronoi
			case _:
				raise ValueError("The specified region map type \'{}\' cannot be identified.".format(which_map))
		axe.axis("off")
		axe.set_title(which_map)
		
		axe.imshow(this.RegionColour[map], interpolation = "nearest", aspect = 1)

regionmap = Regionmap()
print(regionmap)

There are two kinds of regionmap used in our experiment, one generated from uniform random distribution giving regions in high density and variance; whereas the other utilises Voronoi diagram, giving regions in low density and variance.

The random distribution is mainly used for stress test purposes to find the limit of our implementation; the Voronoi diagram is a more realistic model emulating how regions are typically defined in practical use cases.

In [ ]:
def drawRegionmap(rm):
	fig, axe = plt.subplots(1, 2, figsize = (6, 3), constrained_layout = True)
	for a, map in zip(axe, ["Random", "Voronoi"]):
		rm.plotMap(a, map)
	saveFigure(fig, "regionmap")
drawRegionmap(regionmap)

## Test Result

This section contains all information with the test context. Our filename is organised as follows with some named components to distinguiush the test type; You may find a list of all possible named components in the code section.

```md
<category>(<factory>,<filter-tag>,<filter>,<user-tag>).csv
```

Test is written in *ISO C++23* and performed on *Windows 10* compiled using *LLVM Clang 18* with compiler optimisation turned on to max favouring speed. Although all tested filters are parallelisable, the implementation runs each filter on a single thread to ensure fairness.

---

**Category** ::
Test variable being changed whose runtime is then survey. The following test variables are timed:

- Radius of filter kernel
- The number of region on region map
- The number of centroid on Voronoi diagram

**Factory** ::
Generation method to synthesise region map.

- Random : Uniform random distribution varying from $[0,N)$, where $N$ is the region count. The random generator is mainly used to model a worse case scenario to stess test, especially for data structure utilises sparse matrix.
- Voronoi : Voronoi diagram generator with a certain number of centroid. This is to model an average use case, as typically regions are expected to scattered as patches in a confined space.

**Filter Tag** ::
Instruction to filter; this is mainly to specify the internal data structure used. It is mainly structed as `[D|S]C[D|S]H`. In particular:

- *D* stands for *Dense*
- *C* stands for *Cache*
- *S* stands for *Sparse*
- *H* stands for *Histogram*

Hence, filter tag mainly means using dense or sparse matrix on cache and histogram. Note that cache is not applicable to *ReAB*, as it is a brute-force implementation that does not involve use of any filter optimisation.

**Filter** ::
Pretty self-explanatory. The following filters as discussed in the paper are tested:

- Blending by Region Area without Optimisation (*ReAB*)
- Blending by Region Area with Separation and Accumulation (*ReAB+SA*)

**User Tag** ::
An identifier for implementation.

In [ ]:
ALL_CATEGORY = ["Radius", "GlobalRegionCount", "LocalRegionCount"]
ALL_FACTORY = ["Random", "Voronoi"]
ALL_FILTER_TAG = ["DCDH", "DCSH", "SCSH"]
ALL_FILTER = ["ReAB", "ReAB+SA"]
ALL_USER_TAG = ["Default", "Stress"]

ResultFileCache = dict()

# A handy function to create result filename from their name components
def forgeResultFilename(category, factory, filter_tag, filter, user_tag):
	return "{}({},{},{},{}).csv".format(category, factory, filter_tag, filter, user_tag)

def readResult(filename):
	# attempt to load from file cache first
	cached_result = ResultFileCache.get(filename)
	if cached_result is not None:
		return cached_result
	
	result = pd.read_csv(RESULT_DIR + filename, usecols = ["x", "t_median", "memory"])
	ResultFileCache[filename] = result
	return result

Define some helper function to help plotting.

In [ ]:
PLOT_COLOUR = ["tomato", "forestgreen", "dodgerblue"]

def plotAxis(time_axis, memory_axis, category, factory, filter, user_tag):
	time_axis.set_ylabel("Time(ms)")
	memory_axis.set_ylabel("Memory(kB)")
	for colour_idx, filter_tag in enumerate(ALL_FILTER_TAG):
		filename = forgeResultFilename(category, factory, filter_tag, filter, user_tag)
		result = readResult(filename)
		
		time_axis.plot(result["x"], result["t_median"], color = PLOT_COLOUR[colour_idx],
			label = filter_tag + ",Time")
		# the all sparse histograms always use the same memory, so their plots will overlap
		# only need to draw one of them
		if filter_tag != "DCSH":
			# we omit the data structure for cache from tag since it does not contribute to memory usage metric
			memory_axis.plot(result["x"], result["memory"], color = PLOT_COLOUR[colour_idx],
				label = filter_tag[2:] + ",Memory", linestyle = ':')

def plotResult(category, user_tag, factory = ALL_FACTORY, filter = ALL_FILTER, size = (10, 8)):
	fig, axe = plt.subplots(len(factory), len(filter), figsize = size,
		squeeze = False, constrained_layout = True)

	for c, curr_filter in enumerate(filter):
		for r, curr_factory in enumerate(factory):
			curr_axe = axe[r, c]

			time_axis = curr_axe
			memory_axis = curr_axe.twinx()
			
			plotAxis(time_axis, memory_axis, category, curr_factory, curr_filter, user_tag)
			curr_axe.set_title(curr_factory + ',' + curr_filter)
			curr_axe.grid(visible = True, which = "both", axis = "both")

	# just get the last axis for our legend data
	# because the drawing order of all of them is the same
	handle_label = [hl.get_legend_handles_labels() for hl in [time_axis, memory_axis]]
	legend_loc = ["outside upper left", "outside upper right"]
	for (handle, label), loc in zip(handle_label, legend_loc):
		fig.legend(handle, label, loc = loc)

	saveFigure(fig, "result-{}-{}".format(category, user_tag))

## Result with Default Setting

This set of test is run in default setting, i.e., low profile, mainly to form a fair comparison between filters.

### Radius

For most filters, radius of filter kernel determines kernel size and hence affects runtime. This is unarguably the most important benchmark in this paper that demonstrates the effectiveness of filter optimisation and its efficiency against large kernel.

In [ ]:
plotResult("Radius", "Default")

### Global Region Count

In practical application during multi-biome terrain modelling, the total number of biome in the whole world is fixed. However, it is mostly expected that there are only a small subset of these biomes appear in a particular area.

The Voronoi factory can be used to model density of region. Although the region count is varying, the number of centroid is fixed in this test, meaning no matter how many region is defined for the world, there can only be a fixed number of them appeared in the region map.

Consequently, use of a dense matrix results in waste of memory. We shall see a linear complexity (in terms of time and space) with dense matrix and constant with sparse matrix.

The random factory is not applicable in this test case, as it does not provide control towards the number of region presented locally.

In [ ]:
plotResult("GlobalRegionCount", "Default", factory = ["Voronoi"], size = (10, 4))

### Local Region Count

This benchmark is only applicable to Voronoi region factory. The number of centroid is an alternative way to model density of region, apart from varying region count. In this test, the number of region is fixed.

As the total number of region is fixed, the complexity for dense matrix is constant. It is expected to see sparse matrix to be slower than the dense as it would require frequent update to the data structure when the area of each region is small (such as erasure from array list, which is slow).

In [ ]:
plotResult("LocalRegionCount", "Default", factory = ["Voronoi"], size = (10, 4))

## Result with Stress Setting

This set of test is mainly used to investigate the limit of *ReAB+SA*. Other filters are omitted due to extremely long runtime. Only **Radius** category is survey as *ReAB+SA* is optimised specially for large kernel.

In [ ]:
plotResult("Radius", "Stress", factory = ["Random"], filter = ["ReAB+SA"], size = (5, 4))